# XI. Model Inferance

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import joblib
import tensorflow as tf
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import string
import ast
from tensorflow.keras.models import load_model

2023-04-07 15:48:27.516972: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# open chatwords.txt
with open('chatwords.txt') as f:
    data = f.read()
chatwords =  ast.literal_eval(data)
chatwords

# open abbreviation.txt
with open('abbreviation.txt') as abb:
    ab2 = abb.read()
abbreviation =  ast.literal_eval(ab2)
abbreviation

# define stopwords
stop_words = stopwords.words('english')

# define lemmatizer
lem = WordNetLemmatizer()

In [4]:
# Import Functions
def check_chatwords(text):
    temp=[]
    for chat in text.split():
        if chat.upper() in chatwords:
            temp.append(chatwords[chat.upper()])
        else:
            temp.append(chat)
    return " ".join(temp)

def lower(text):
    data = text.lower()
    return data 

def check_abbr(text):
    temp2=[]
    for abbr in text.split():
      if abbr in abbreviation:
          temp2.append(abbreviation[abbr])
      else:
          temp2.append(abbr)

    return " ".join(temp2)

def check_punctuation(text):
    data = re.sub("[^a-zA-Z]",' ', text)
    data = re.sub('[[^]]*]', ' ', data)
    data = re.sub(r"\n", " ", data)
    data = data.strip()
    data = ' '.join(data.split())
    return data

def token_stopwords_lemma(text):
    tokens = word_tokenize(text)
    stop_words2 = ' '.join([word for word in tokens if word not in stop_words])
    data = [lem.lemmatize(word) for word in stop_words2.split()]
    data = ' '.join(data)
    return data

In [5]:
# Load Model
final_gru= tf.keras.models.load_model('model_gru')

2023-04-07 15:53:45.802603: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-04-07 15:53:45.802928: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2023-04-07 15:53:45.803304: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [7]:
# Membuat DataFrame baru
df_inf = ["Dont show yourself. How far. Put new pictures up on facebook", "Someone has conacted our dating service and entered your phone because they fancy you!To find out who it is call from landline 09111030116. PoBox12n146tf15"]

In [10]:
# convert to dataframe
df_inf1 = pd.DataFrame()
df_inf1['message'] = df_inf

In [11]:
df_inf1['message'] = df_inf1['message'].apply(lambda j: check_chatwords(j))
df_inf1['message'] = df_inf1['message'].apply(lambda k: lower(k))
df_inf1['message'] = df_inf1['message'].apply(lambda v: check_abbr(v))
df_inf1['message'] = df_inf1['message'].apply(lambda r: check_punctuation(r))
df_inf1['message'] = df_inf1['message'].apply(lambda m: token_stopwords_lemma(m))

/var/folders/ts/drf61cb133z5gjz33hcxvy280000gn/T/ipykernel_22585/3669643084.py:27: FutureWarning: Possible nested set at position 1
  data = re.sub('[[^]]*]', ' ', data)


In [14]:
y_pred_inf = final_gru.predict(df_inf1['message'])
y_pred_inf = np.where(y_pred_inf >= 0.5, 1, 0)
y_pred_inf

1/1 [==============================] - 3s 3s/step


array([[0],
       [1]])

In [15]:
# Membuat dataframe dari array
pred_df = pd.DataFrame(y_pred_inf, columns=['label'])
pred_df

,label
0,0
1,1


In [18]:
# Melakukan prediksi pada dataframe yang baru dibuat
df_inf2 = pd.DataFrame(df_inf,columns=['message'])
df_combined = pd.concat([df_inf2,pred_df], axis=1)
df_combined

,message,label
0,Dont show yourself. How far. Put new pictures ...,0
1,Someone has conacted our dating service and en...,1


- Berdasarkan model inference tersebut, dapat disimpulkan bahwa model telah dilatih dengan menggunakan dataset SMS Spam Collection Dataset untuk melakukan klasifikasi pesan menjadi dua kategori, yaitu spam (label 1) dan non-spam (label 0). Model berhasil melakukan klasifikasi pada dua pesan yang diberikan sebagai contoh, dengan pesan pertama diklasifikasikan sebagai non-spam dan pesan kedua diklasifikasikan sebagai spam.